In [9]:
import pandas as pd
import numpy as np
import pydantic as py
import json
import re
import random
from string import Formatter
from pydantic import BaseModel, Field, parse_obj_as

In [8]:
!pip3 install "pydantic==1.*"

You should consider upgrading via the '/Users/anujatayal/pptod/bin/python3 -m pip install --upgrade pip' command.


In [10]:
class FoodInputVariables(BaseModel):
    food: str = None
    part: str = None
    cook: str = None
    animal: str = None
    # cuisine: str = None
    foodWeight: str = None
    metric: str = None
    nutrient: str = None
    # texture: str = None
    # color: str = None
    # flavor: str = None
    ingredient: str = None
    type: str = None

class FoodOutputVariables(BaseModel):
    saltmetric: str = None
    value: str = None
    defaultValue: str = None

FoodDataset = py.create_model("FoodDataset", __base__=(FoodInputVariables, FoodOutputVariables))

In [13]:
df=pd.read_csv('dataset_full_pptod.csv')
# df_qa=pd.DataFrame(columns=['food','part','cook','animal','cuisine','shouldertype','groundtype','bonetype','saucetype','foodWeight'])


/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/2033112259.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('dataset_full_pptod.csv')


In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
# df['foodDescription'] = df['foodDescription'].replace(r'[^a-zA-Z%-/ ]', '', regex=True) 
# df['foodDescription'] = df['foodDescription'].replace(r'andor', ',', regex=True) 
# df['foodDescription'] = df['foodDescription'].replace(r',', '', regex=True) 
# df['foodDescription'] = df['foodDescription'].replace(r'--', ',', regex=True) 
# df['metric'] = df['metric'].replace(r'[()]', '', regex=True) 



In [ ]:
# f = open('db/ontology-animal.json')
# data = json.load(f)

In [ ]:
# corpus=[]
# for description in df['foodDescription']:
#   ingredients=description.split()
#   corpus.append([i.lower()  for i in ingredients])
# corpus[:5]

In [ ]:
# df=df.reindex(columns=df.columns.tolist()+['food','animal','part','cook','cuisine','type'])

In [ ]:
# key_final=[i.split("-")[1] for i in data.keys()]

In [ ]:
# i=0
# for food in corpus:
#   # print(food)
#   for items in food:
#     for key in data:
#       if items in data[key]:
#         key_item=key.split("-")[1]
#         # print(key_item)
#         if key_item!='foodWeight' and not i in df.index or pd.isna(df[key_item][i]):
#           df.at[i,key_item]=str(items)
#         else:
#           # print(data[key].index(df_qa.loc[i][key_item]),data[key].index(items),items,df_qa.loc[i][key_item])
#           if key_item!='foodWeight' and data[key].index(df.loc[i][key_item])<data[key].index(items):
#             df.at[i,key_item]=str(items)
#   i+=1
# df['nutrient']='salt'

In [14]:
df1=df[['food','part','cook','animal','type','ingredient','nutrient']]

In [15]:
df1.head()
df1 = df1.applymap(eval)
df1 = df1.applymap(lambda x:' '.join(x[::-1]))
df1 = df1.applymap(lambda x:re.sub('_', ' ', x))
df1 = pd.concat([df1, df[['foodWeight','metric','saltmetric','value','defaultValue','foodDescription']]], axis=1)
df1 = df1.applymap(lambda x: re.sub('\s+', ' ', x) if isinstance(x, str) else x)
df1 = df1.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/1213709854.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(eval)
/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/1213709854.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x:' '.join(x[::-1]))
/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/1213709854.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x:re.sub('_', ' ', x))
/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/1213709854.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1.applymap(lambda x: re.sub('\s+', ' ', x) if isinstance(x, str) else x)
/var/folders/l7/r3k40z2d7cz01382wp926w8h0000gn/T/ipykernel_84802/1213709854.py:7: FutureWarning: DataFrame.applymap has been deprecate

In [16]:
df1=df1.drop_duplicates()
df1=df1.reset_index(drop=True)

In [ ]:
print(len(df))
print(len(df1))
# print(len(df_dup))

In [17]:
df_dup=df1[~df1.duplicated(['food','part','cook','animal','type','ingredient','nutrient','foodWeight','metric','saltmetric','value','defaultValue'])]
df_dup=df_dup.reset_index(drop=True)

In [18]:
df_dup['ingredient'].unique()

array(['', 'with added vitamin d', 'added calcium',
       'with high vitamin c and added calcium',
       'with carrots and tomatoes and or tomato based sauce',
       'with cream sauce and seafood', 'with added butter',
       'made from no sugar added dry mix with whole milk',
       'without added sodium', 'with added nutrients',
       'without added sugar', 'without added ascorbic acid',
       'fortified with vitamin d',
       'without added vitamin a and vitamin d', 'without added calcium',
       'added ascorbic acid', 'high vitamin c and added thiamin',
       'vitamins b and b',
       'with added vitamin d and without added vitamin a',
       'multigrain cereal and added iron fortified',
       'with added potassium', 'with added solution cooked',
       'with vitamin c added',
       'with high vitamin c with other added vitamins'], dtype=object)

In [19]:
db=[]
for i in range(len(df_dup)):
    db_i={}
    db_i['value']=str(df_dup.loc[i]['value'])
    db_i['foodDescription']=str(df_dup.loc[i]['foodDescription'])
    db_i['saltmetric']=str(df_dup.loc[i]['saltmetric'])
    db_i['metric']=str(df_dup.loc[i]['metric'])
    db_i['foodweight']=str(df_dup.loc[i]['foodWeight'])
    db.append(db_i)

In [ ]:
len(db)

In [ ]:
json_db= json.dumps(db,indent=4)
# data=json.dumps(query,indent=4)
with open("db.json","w") as file:
    file.write(json_db)


In [60]:
df_final.iloc[77733]

food                   restaurant
part                             
cook                             
animal                           
type            lasagna with meat
ingredient                       
nutrient                     salt
foodWeight                  100.0
metric                          g
saltmetric                     mg
value                       466.0
defaultValue                466.0
Name: 77733, dtype: object

In [63]:
df_final.iloc[77734]

food                   restaurant
part                             
cook                             
animal                           
type            lasagna with meat
ingredient                       
nutrient                     salt
foodWeight                  457.0
metric                          g
saltmetric                     mg
value                      2130.0
defaultValue                466.0
Name: 77734, dtype: object

In [76]:
matching_rows = df_final[df_final['food']=='restaurant']
    # df_final['food'], df_final['type'] = df_final['type'].copy(), df_final['food'].copy()


In [78]:
for index,row in matching_rows.iterrows():
    df_final.at[index,'food'],df_final.at[index,'type'] = row['type'],row['food']

In [79]:
df_final[df_final['food']=='restaurant']

,food,part,cook,animal,type,ingredient,nutrient,foodWeight,metric,saltmetric,value,defaultValue


In [70]:
df_final[df_final['food']=='restaurant']

,food,part,cook,animal,type,ingredient,nutrient,foodWeight,metric,saltmetric,value,defaultValue
171,restaurant,,,,sweet and sour pork,,salt,100,g,mg,304.0,304.0
172,restaurant,,,,sweet and sour pork,,salt,41.8,g,mg,127.0,304.0
173,restaurant,,,,sweet and sour pork,,salt,3,pieces,mg,127.0,304.0
174,restaurant,,,,sweet and sour pork,,salt,611,g,mg,1860.0,304.0
175,restaurant,,,,sweet and sour pork,,salt,1,order,mg,1860.0,304.0
...,...,...,...,...,...,...,...,...,...,...,...,...
87000,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,2.0,taco,mg,1420.0,509.0
87001,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,134.0,g,mg,682.0,509.0
87002,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,1.0,taco,mg,682.0,509.0
87003,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,281.0,g,mg,1430.0,509.0


In [68]:
df_final[df_final['food']=='restaurant']

,food,part,cook,animal,type,ingredient,nutrient,foodWeight,metric,saltmetric,value,defaultValue
171,restaurant,,,,sweet and sour pork,,salt,100,g,mg,304.0,304.0
172,restaurant,,,,sweet and sour pork,,salt,41.8,g,mg,127.0,304.0
173,restaurant,,,,sweet and sour pork,,salt,3,pieces,mg,127.0,304.0
174,restaurant,,,,sweet and sour pork,,salt,611,g,mg,1860.0,304.0
175,restaurant,,,,sweet and sour pork,,salt,1,order,mg,1860.0,304.0
...,...,...,...,...,...,...,...,...,...,...,...,...
87000,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,2.0,taco,mg,1420.0,509.0
87001,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,134.0,g,mg,682.0,509.0
87002,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,1.0,taco,mg,682.0,509.0
87003,restaurant,,,,cheese and lettuce soft taco with ground beef,,salt,281.0,g,mg,1430.0,509.0


df_final[77734]

In [20]:
df_final=df_dup[['food','part','cook','animal','type','ingredient','nutrient','foodWeight','metric','saltmetric','value','defaultValue']]

In [80]:
def normalize(foodDataset: FoodDataset) -> FoodDataset:
    foodVariables = foodDataset.copy()
    if (foodDataset.food is None):
        foodVariables.food = foodDataset.animal
        foodVariables.animal = None
    elif (foodDataset.animal != None and foodDataset.food.strip().lower() == foodDataset.animal.strip().lower()):
        foodVariables.animal = None
    return foodVariables

tempFoodDescriptions = [(normalize(FoodDataset.parse_obj(row))) for index, row in df_final.replace('', None).iterrows() ]
foodDescriptions = [foodDataset for foodDataset in tempFoodDescriptions if foodDataset.food is not None]

In [81]:
foodText={"What is the {goal.nutrient} content in {goal.food} ?",
"how much {goal.nutrient} in {goal.food} ?",
"sodium content in {goal.food} ?",
"{goal.food} {goal.nutrient}",
"what is the nutritional composition of {goal.food} ?",
"Can I eat {goal.food} ?",
"is it safe for me to eat {goal.food} ?",
"Can heart failure patients eat {goal.food} ?",
"how much {goal.food} is safe for me to eat ?",
"how much {goal.food} can I eat ?",
"is it okay to eat {goal.food} ?",
"what type of {goal.food} can I eat ?",
"can I eat {goal.food} after I eat my medicine ?",
"can i eat {goal.food} if I have a bad heart ?",
"can i eat {goal.food} with my condition ?",
"can my partner with heart issues eat {goal.food} ?",
"can my partner with heart failure eat {goal.food} ?",
"is {goal.food} healthy ?",
"can people with bad heart eat {goal.food} ?",
"can I eat {goal.food} with medicine ?",
"is {goal.food} okay for heart patients ?",
"what is the best way to eat {goal.food} ?",
"I would like to cook some {goal.food} tonight",
"I am making {goal.food} tonight"
""
}
foodCookText={
"What is the {goal.nutrient} content in {goal.cook} {goal.food} ?",
}
foodWeightText={
"how much {goal.nutrient} in {goal.foodWeight} {goal.metric} of {goal.food} ?",
}

In [82]:
foodMeat={"what type of meat are you eating ?",
"what kind of {goal.food} ?",
"what type of meat is it ?",
"what type of {goal.food} are you eating today ?",
}

def randomMeatPrompt() -> str:
    return random.choice(tuple(foodMeat))

def getMeatPrompt(goal: FoodInputVariables) -> str:
    return randomMeatPrompt().format(goal = goal)
    

In [83]:
def randomFoodTextPrompt(goal: FoodInputVariables) -> str:
    prompts = set(foodText)
    if (goal.cook is not None):
        prompts = prompts.union(foodCookText)
    if (goal.foodWeight is not None and goal.metric is not None):
        prompts = prompts.union(foodWeightText)
    return random.choice(tuple(prompts))

def getFoodTextPrompt(goal: FoodInputVariables) -> str:
    return randomFoodTextPrompt(goal).format(goal = goal)



In [84]:
foodCook={"how is {goal.food} cooked ?",
"how are you cooking your {goal.food} ?",
"how do you plan to cook your {goal.food} ?",
"can you tell me how your {goal.food} is cooked ?",
"do you know how {goal.food} is cooked ?"}

def randomCookPrompt() -> str:
    return random.choice(tuple(foodCook))

def getCookPrompt(goal: FoodInputVariables) -> str:
    return randomCookPrompt().format(goal = goal)

In [85]:
foodCuisine={"{goal.cuisine} ?",
"what is the cuisine of {goal.food} ?",
"Are you going for a particular style of cuisine tonight?",
"What's your favorite type of cuisine with {goal.food} to order when you eat?"

}

def randomCuisinePrompt() -> str:
    return random.choice(tuple(foodCuisine))

def getCuisinePrompt(goal: FoodInputVariables) -> str:
    return randomCuisinePrompt().format(goal = goal)

In [86]:
foodType={"what type of {goal.food} are you eating ?",
"what kind of {goal.food} ?",
"can you tell me more information about {goal.food} ?",
"what is your choice of {goal.food} today",
"Can you tell me what {goal.food} are you enjoying ?",
"Do you know what kind of {goal.food} are you having ?",
"Can you describe your {goal.food} for me ?",
"What is your preference when it comes to {goal.food} ?",
"Can you describe the type of {goal.food} you are having ?",
"Are you going for a particular type of {goal.food} ?",
"Could you share what is in your {goal.food} ?",
"What kind of {goal.food} did you decide on eating ?",
}

def randomTypePrompt() -> str:
    return random.choice(tuple(foodType))

def getTypePrompt(goal: FoodInputVariables) -> str:
    return randomTypePrompt().format(goal = goal)

In [87]:
foodPart={"And which part of {goal.food} are you eating ?",
"part of {goal.food} ?",
"can you tell which part of {goal.food} are you eating?"}
def randomPartPrompt() -> str:
    return random.choice(tuple(foodPart))

def getPartPrompt(goal: FoodInputVariables) -> str:
    return randomPartPrompt().format(goal = goal)

In [88]:
foodTypeAns={"I am eating {goal.type} {goal.food}",
"{goal.type} {goal.food}",
"{goal.type}",
"it is {goal.type}",
"its {goal.type}",
"The {goal.food} is {goal.type}",
"My meal consists of {goal.type} {goal.food}",
"I've got some delicious {goal.type} {goal.food} on my plate",
"I decided to cook up some {goal.type} {goal.food} for dinner.",
"Tonight, it's all about {goal.type} {goal.food}",
"{goal.type} {goal.food} are on the menu for me tonight.",
"It's a hearty meal with {goal.type} {goal.food}",
"{goal.type} {goal.food} are what's on my plate at the moment."

}

def randomTypeAnsPrompt() -> str:
    return random.choice(tuple(foodTypeAns))

def getTypeAnsPrompt(goal: FoodInputVariables) -> str:
    return randomTypeAnsPrompt().format(goal = goal)

In [89]:
foodPartAns={"I am eating {goal.part} {goal.food} .",
"{goal.part}",
"it is {goal.part}"
}
def randomPartAnsPrompt() -> str:
    return random.choice(tuple(foodPartAns))

def getPartAnsPrompt(goal: FoodInputVariables) -> str:
    return randomPartAnsPrompt().format(goal = goal)

In [90]:
foodCookAns={"it is {goal.cook}",
"{goal.cook}",
"its {goal.cook}",
"The {goal.food} is {goal.cook}",
"it is {goal.cook} {goal.food}"
}

def randomCookAnsPrompt() -> str:
    return random.choice(tuple(foodCookAns))

def getCookAnsPrompt(goal: FoodInputVariables) -> str:
    return randomCookAnsPrompt().format(goal = goal)

In [91]:
foodWeightAns={"it is around {goal.foodWeight} {goal.metric}",
"it is about {goal.foodWeight} {goal.metric}",
"{goal.foodWeight} {goal.metric} .",
"its {goal.foodWeight} {goal.metric}",
"dont know",
"i dont understand",
"I am having {goal.foodWeight} {goal.metric} of {goal.food}",
"I like having {goal.foodWeight} {goal.metric} of {goal.food}",
"{goal.food} is around {goal.foodWeight} {goal.metric}",
"The {goal.food} weighs approximately {goal.foodWeight} {goal.metric}",
"You'll find that the {goal.food} is around {goal.foodWeight} {goal.metric} in weight.",
"You're looking at a {goal.food} with a weight of approximately {goal.foodWeight} {goal.metric}"
}

def randomWeightAnsPrompt() -> str:
    return random.choice(tuple(foodWeightAns))

foodWeight={"what is the quantity of {goal.food} ?",
"how much {goal.food} are you using today ?",
"how much {goal.food} are you eating ?",
"What your {goal.food} portion like ? ",
"Can you share what is the size of {goal.food} are you eating ?",
"what is the serving size for the {goal.food} ?" 
}

def randomWeightPrompt() -> str:
    return random.choice(tuple(foodWeight))

finalAns={"{goal.food} has {output.value} {output.saltmetric} of {goal.nutrient} .",
          "you will find about {output.value} {output.saltmetric} of {goal.nutrient} in {goal.food}",
          "The {goal.nutrient} content in {goal.food} is roughly {output.value} {output.saltmetric}. ",
          "You can expect to have approximately {output.value} {output.saltmetric} of {goal.nutrient} in that {goal.food}.",
          "That {goal.food}'s {goal.nutrient} level is at {output.value} {output.saltmetric}",
          "You'll be consuming around {output.value} {output.saltmetric} of {goal.nutrient} with that {goal.food}.",
          "That {goal.food} contains a modest {output.value} {output.saltmetric} of {goal.nutrient}."
}

def randomFinalPrompt() -> str:
    return random.choice(tuple(finalAns))

In [92]:
#annotation
class Dialog(BaseModel):
    goal: dict = None
    log: list = None

class LogItem(BaseModel):
    text: str = None
    metadata: dict = {}

class AnnotatedLogItem(BaseModel):
    text: str = ""
    metadata: dict = {}
    dialog_act: dict = {}
    span_info: list[list] = []

class SystemAnnotation(BaseModel):
    book = FoodOutputVariables(value="", saltmetric="")
    semi = FoodInputVariables()

In [93]:
def getAnnotations(prompt, goal, exclude = set()):
    goalVariables = [fn for _, fn, _, _ in Formatter().parse(prompt) if fn is not None]
    variables = [i.split(".")[1] for i in goalVariables if i not in exclude]
    return [([i, goal.__getattribute__(i)]) for i in variables]
    
def toDict(annotations: list[list]) -> dict:
    return {i[0]:i[1] for i in annotations}

In [94]:
def getSpanInfo(prompt: str, goal: FoodInputVariables, output: FoodDataset = None):
    outputKeys = set(output.dict().keys()).difference(set(goal.dict().keys())) if output else {}
    text = prompt.format(goal = goal, output = output)
    annotations = getAnnotations(prompt, output or goal)
    items = []
    for key, value in annotations:
        type = "Booking-Book" if key in outputKeys else "food-Inform"
        startCharacterIndex = text.find(value)
        startWordIndex = text.count(' ', 0, startCharacterIndex)
        endWordIndex = startWordIndex + value.count(' ')
        item = (type, key, value, startWordIndex, endWordIndex)
        items.append(item)
    return items


In [95]:
def getBooking(prompt, goal):
    goalVariables = [fn for _, fn, _, _ in Formatter().parse(prompt) if fn is not None]
    variables = [i.split(".")[1] for i in goalVariables if i in ['output.value','output.saltmetric']]
    return [([i, goal.__getattribute__(i)]) for i in variables]

In [96]:
def testGetSpanOutput():
    my_random_goal=normalize(FoodDataset())
    my_random_goal.food='my pork'
    my_random_goal.cook=None 
    my_random_goal.value='100.0' 
    my_random_goal.saltmetric='g' 
    my_random_goal.nutrient='rock salt' 
    
    prompt = "{goal.food} has {output.value} {output.saltmetric} of {goal.nutrient}"
    print(prompt)
    return getBooking(prompt,my_random_goal)
    # return getSpanOutput(prompt, my_random_goal)

testGetSpanOutput()

{goal.food} has {output.value} {output.saltmetric} of {goal.nutrient}


[['value', '100.0'], ['saltmetric', 'g']]

In [50]:
def testGetSpanInfo():
    my_random_goal=normalize(FoodDataset())
    my_random_goal.food='my pork'
    my_random_goal.cook='raw' 
    my_random_goal.type='lettuce' 
    my_random_goal.foodWeight='100.0' 
    my_random_goal.metric='g' 
    my_random_goal.nutrient='rock salt' 
    # prompt = randomFoodTextPrompt(my_random_goal)
    prompt = randomTypeAnsPrompt()
    print(prompt)
    return getSpanInfo(prompt, my_random_goal)

testGetSpanInfo()

{goal.type} {goal.food} are what's on my plate at the moment.


[('food-Inform', 'type', 'lettuce', 0, 0),
 ('food-Inform', 'food', 'my pork', 1, 2)]

In [97]:
#annotated
def annotateFoodPrompt(goal: FoodInputVariables, annotation: SystemAnnotation) -> list[AnnotatedLogItem]:
    logs = []
    if goal.food is not None:
        prompt = randomFoodTextPrompt(goal)
        annotations = getAnnotations(prompt, goal)
        userRequest = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act = {"food-Request": [
                ["value", "?"],
                ["saltmetric","?"]
            ], "food-Inform": annotations},
            span_info=getSpanInfo(prompt, goal)
            )
        logs.append(userRequest)
        for key, value in toDict(annotations).items():
            annotation.semi.__setattr__(key, value)
        annotation.semi.nutrient="salt"
        logs += annotateAnimalPrompt(goal, annotation)
    return logs

def annotateAnimalPrompt(goal: FoodInputVariables, annotation = SystemAnnotation) -> list[AnnotatedLogItem]:
    if goal.animal is not None and annotation.semi.animal == "":
        systemRequest = AnnotatedLogItem(
            text = randomMeatPrompt().format(goal = goal),
            dialog_act = {"food-Request": [
                ["animal", "?"],
            ]},
            metadata = {
                "food": annotation.dict(exclude_none = True)
            })
        prompt="i am eating {goal.animal} {goal.food} ."
        userResponse = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act={"food-Inform":[
                ["animal", goal.animal]
            ]},
            span_info=getSpanInfo(prompt, goal)
        )
        annotation.semi.animal = goal.animal
        return [systemRequest,userResponse]
    return []

def annotateCookPrompt(goal: FoodInputVariables, annotation: SystemAnnotation) -> list[AnnotatedLogItem]:
    if goal.cook is not None and goal.cook != 'raw' and annotation.semi.cook == "":
        systemRequestCook = AnnotatedLogItem(
            text = getCookPrompt(goal),
            dialog_act = {"food-Request": [
                ["cook", "?"],
            ]},
            metadata = {
                "food": annotation.dict(exclude_none = True)
            })
        prompt=randomCookAnsPrompt()
        userResponseCook = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act={"food-Inform": [
                ["cook",goal.cook]
            ]},
            span_info=getSpanInfo(prompt, goal)

        )
        annotation.semi.cook = goal.cook
        return [systemRequestCook,userResponseCook]
    return []

def annotateTypePrompt(goal: FoodInputVariables, annotation: SystemAnnotation) -> list[AnnotatedLogItem]:
    if goal.type is not None and annotation.semi.type == "":
        prompt = randomTypePrompt()
        systemRequestType = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act = { "food-Request": [
                ["type", "?"],
            ]},
            metadata = {
                "food": annotation.dict(exclude_none = True)
            })
        prompt=randomTypeAnsPrompt()
        userResponseType = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act={ "food-Inform": [
                ["type",goal.type]
            ]},
            span_info=getSpanInfo(prompt, goal)

        )
        annotation.semi.type = goal.type
        return [systemRequestType,userResponseType]
    return []

def annotatePartPrompt(goal: FoodInputVariables, annotation: SystemAnnotation) -> list[AnnotatedLogItem]:
    if goal.part is not None and annotation.semi.part == "":
        prompt = randomPartPrompt()
        # print("prompt part ques",prompt)
        systemRequest = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act = { "food-Request": [
                ["part", "?"],
            ]},
            metadata = {
                "food": annotation.dict(exclude_none = True)
            })
        prompt=randomPartAnsPrompt()
        # print("prompt part ans",prompt)
        userResponse = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act={ "food-Inform": [
                ["part",goal.part]
            ]},
            span_info=getSpanInfo(prompt, goal)

        )
        annotation.semi.part = goal.part
        return [systemRequest,userResponse]
    return []

def annotateWeightPrompt(goal: FoodInputVariables, annotation: SystemAnnotation,flag) -> list[AnnotatedLogItem]:
    if goal.foodWeight is not None and annotation.semi.foodWeight == "":
        prompt=randomWeightPrompt()
        systemRequest = AnnotatedLogItem(
            text = prompt.format(goal = goal),
            dialog_act = { "food-Request": [
                ["foodWeight", "?"],
                ["metric","?"]
            ]},
            metadata = {
                "food": annotation.dict(exclude_none = True)
            })
        prompt=randomWeightAnsPrompt()
        if prompt not in ['dont know','i dont understand']:
            userResponse = AnnotatedLogItem(
                text = prompt.format(goal = goal),
                dialog_act={ "food-Inform": [
                    ["foodWeight",goal.foodWeight],
                    ["metric",goal.metric]
                ]},
                span_info=getSpanInfo(prompt, goal)

            )
            annotation.semi.foodWeight = goal.foodWeight
            annotation.semi.metric = goal.metric
        else:
            flag=0
            userResponse = AnnotatedLogItem(
                text = prompt.format(goal = goal),
                dialog_act={ "food-Inform": [
                    ["foodWeight","100"],
                    ["metric","g"]
                ]},
                span_info=getSpanInfo(prompt, goal)

            )
            annotation.semi.foodWeight = "100"
            annotation.semi.metric = "g"
        # print(prompt,userResponse)
        return [systemRequest,userResponse,flag]
    return []

def annotateFinalPrompt(goal: FoodInputVariables, annotation: SystemAnnotation, output: FoodOutputVariables,flag) -> list[AnnotatedLogItem]:
    prompt = randomFinalPrompt()
    if flag == 0:
        output.value = output.defaultValue
        goal.foodWeight = "100"
        goal.metric = "g"
    annotation.book.value = output.value
    annotation.book.saltmetric = output.saltmetric
    systemResponse = AnnotatedLogItem(
        text = prompt.format(goal = goal, output = output),
        dialog_act={ "food-Inform": getAnnotations(prompt, goal, {"output.value", "output.saltmetric"}),
                            "Booking-Book": getBooking(prompt, output)},
                    span_info=getSpanInfo(prompt, goal, output),
        metadata = {
            "food": annotation.dict(exclude_none = True)
    })
    return [systemResponse]
    
def annotatePrompts(foodDataset: FoodDataset) -> Dialog:
    logs = []
    goal = FoodInputVariables.parse_obj(foodDataset.dict())
    annotation = SystemAnnotation()
    for key in goal.dict(exclude_none=True).keys():
        annotation.semi.__setattr__(key, "")
    flag=1
    logs += annotateFoodPrompt(goal, annotation)
    logs += annotateTypePrompt(goal, annotation)
    logs += annotateCookPrompt(goal, annotation)
    logs += annotatePartPrompt(goal, annotation)
    response = annotateWeightPrompt(goal, annotation,flag)
    # print(len(response))
    if len(response)>0: 
        logs += response[:2]
        flag = response[2]
        # print(logs)
    else:
        logs += response
    logs += annotateFinalPrompt(goal, annotation, foodDataset,flag)
    output = FoodOutputVariables.parse_obj(foodDataset.dict())
    return Dialog(goal = {
        "food": {
            "info": goal.dict(exclude_none=True),
            "book":output.dict(exclude_none=True,exclude={'defaultValue'})
        }
    }, log = logs)
    
def toPlain(dialog: Dialog) -> Dialog:
    return Dialog(goal = dialog.goal,
                  log = [LogItem(text= i.text) for i in dialog.log])

annotated_queries = {}
queries = {}
for i, foodDataset in enumerate(foodDescriptions):
    key = "SNG" + str(i) + ".json"
    annotated_value = annotatePrompts(normalize(foodDataset))
    annotated_queries[key] = annotated_value
    queries[key] = toPlain(annotated_value)


In [98]:
query={k:v.dict() for k,v in queries.items()}
data=json.dumps(query,indent=4)
with open("pptod_food/data/final-dataset/data.json","w") as file:
    file.write(data)


In [99]:
annotated_query={k:v.dict(exclude_none=True, by_alias=True) for k,v in annotated_queries.items()}
annotated_data=json.dumps(annotated_query,indent=4)
with open("pptod_food/data/final-dataset/annotated_data.json","w") as file:
    file.write(annotated_data)

In [ ]:
short_queries=list(queries)[:100]
query={k:queries[k].dict() for k in short_queries}
data=json.dumps(query,indent=4)
with open("data_100.json","w") as file:
    file.write(data)

In [ ]:
short_anno_queries=list(annotated_queries)[:100]
annotated_query={k:annotated_queries[k].dict(exclude_none=True, by_alias=True) for k in short_anno_queries}
annotated_data=json.dumps(annotated_query,indent=4)
with open("annotated_user_da_with_span_full_100.json","w") as file:
    file.write(annotated_data)

In [ ]:
i=0
test=""
val=""
while i<100:
    i+=10
    test+=f"SNG{i}.json\n"
    i+=1
    val+=f"SNG{i}.json\n"

In [ ]:
f=open("testListFile.json","w")
f.write(test)
# f1=open("valListFile.json","w")
# f1.write(val)


In [ ]:
x=['food-nutrient','food-food']
a=[i.split('-')[1] for i in x]

In [ ]:
a.extend(['foodweight','metric'])


In [ ]:
import re
text='how much [value_food] are you eating ?'
# re.finditer(r'(\[value_)', text)
[(m.start(0), m.end(0)) for m in re.finditer(r'(\[value_)', text)]


In [ ]:
words=re.findall(r'\bvalue_\w*', text)

In [ ]:
[word.split('_')[1] for word in words]

In [75]:
import pandas as pd

# Create a sample DataFrame
data = {
    'Column1': [10, 20, 30, 40],
    'Column2': [50, 60, 70, 80],
    'ConditionColumn': ['A', 'B', 'A', 'B']
}

df = pd.DataFrame(data)
print(df)
# Define the condition for swapping columns
condition_value = 'A'

# Find the rows where the condition is met
matching_rows = df[df['ConditionColumn'] == condition_value]

# Swap columns within the matching rows
for index, row in matching_rows.iterrows():
    df.at[index, 'Column1'], df.at[index, 'Column2'] = row['Column2'], row['Column1']

print(df)

   Column1  Column2 ConditionColumn
0       10       50               A
1       20       60               B
2       30       70               A
3       40       80               B
   Column1  Column2 ConditionColumn
0       50       10               A
1       20       60               B
2       70       30               A
3       40       80               B


In [73]:
data

{'Column1': [10, 20, 30, 40],
 'Column2': [50, 60, 70, 80],
 'ConditionColumn': ['A', 'B', 'A', 'B']}